In [2]:
# PART 1: IMPORTS AND SETUP

# Standard data manipulation (from Week 1-2)
import pandas as pd
import numpy as np

# Geospatial libraries (from Week 3-5)
import geopandas as gpd
from shapely.geometry import Point, Polygon, box

# Network analysis (from Week 7)
import osmnx as ox

# Raster data (from Week 6)
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats

# Visualization (from Week 3)
import matplotlib.pyplot as plt
import seaborn as sns

# For file management
import os
from pathlib import Path

# Set display options to see more data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set figure style (like in class)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


In [3]:
# ============================================================================
# PART 6: GET OFFICIAL SEPTA TRANSIT STOPS WITH CONNECTIVITY ANALYSIS
# ============================================================================

import json

print("\n" + "="*70)
print("LOADING OFFICIAL SEPTA STOPS DATA")
print("="*70)

# Load the official SEPTA GeoJSON
json_path = 'data/raw/Transit_Stops_(Spring_2025).geojson'

print(f"Loading from: {json_path}")
septa_all = gpd.read_file(json_path)

print(f"✓ Loaded {len(septa_all):,} SEPTA stops system-wide")

# Filter to University City study area
septa_in_area = gpd.sjoin(septa_all, study_area, how='inner', predicate='intersects')
septa_in_area = septa_in_area.drop_duplicates(subset=['StopId'])
if 'index_right' in septa_in_area.columns:
    septa_in_area = septa_in_area.drop(columns=['index_right'])

print(f"✓ Found {len(septa_in_area)} stops in University City")

# Categorize by transit type
def categorize_septa_stop(line_abbr):
    line = str(line_abbr).upper()
    rail_lines = ['AIR', 'CHE', 'CHW', 'CYN', 'FOX', 'LAN', 'MED', 'NOR', 
                  'PAO', 'TRE', 'WAR', 'WIL', 'WTR']
    subway_lines = ['MFL', 'BSL', 'BSS', 'NHSL']
    trolley_lines = ['10', '11', '13', '34', '36', '101', '102', 'GRHL']
    
    if line in rail_lines:
        return 'Regional Rail'
    elif line in subway_lines:
        return 'Subway'
    elif line in trolley_lines:
        return 'Trolley'
    else:
        return 'Bus'

septa_in_area['transit_type'] = septa_in_area['LineAbbr'].apply(categorize_septa_stop)

print("\nStops by type:")
for transit_type, count in septa_in_area['transit_type'].value_counts().items():
    print(f"  {transit_type}: {count}")

# Get major transit stops (Subway - Market-Frankford Line)
major_transit = septa_in_area[septa_in_area['transit_type'] == 'Subway'].copy()
major_stations_unique = major_transit.groupby('StopName').first().reset_index()
major_stations_gdf = gpd.GeoDataFrame(
    major_stations_unique,
    geometry=major_stations_unique['geometry'],
    crs=septa_in_area.crs
)

print(f"\n✓ Major transit stations: {len(major_stations_gdf)}")
for idx, row in major_stations_gdf.iterrows():
    print(f"  • {row['StopName']}")

# Add nearest bus stops for connectivity analysis
print("\n" + "-"*70)
print("ADDING CONNECTING BUS STOPS FOR TRANSIT CONNECTIVITY")
print("-"*70)

n_bus_stops = 10  # Number of bus stops per major station

bus_stops = septa_in_area[septa_in_area['transit_type'] == 'Bus'].copy()

# Project for distance calculations
major_proj = major_stations_gdf.to_crs('EPSG:2272')
bus_proj = bus_stops.to_crs('EPSG:2272')

selected_bus_stops = []

for idx, station in major_proj.iterrows():
    bus_proj['distance'] = bus_proj.geometry.distance(station.geometry)
    nearest = bus_proj.nsmallest(n_bus_stops, 'distance').copy()
    nearest['connects_to'] = station['StopName']
    selected_bus_stops.append(nearest)

all_selected_bus = pd.concat(selected_bus_stops, ignore_index=True)
unique_bus_stops = all_selected_bus.drop_duplicates(subset=['StopId']).to_crs('EPSG:4326')

print(f"✓ Selected {len(unique_bus_stops)} connecting bus stops")

# Combine major transit + bus stops
major_transit['stop_category'] = 'Major Transit'
unique_bus_stops['stop_category'] = 'Feeder Bus'

cols_common = ['StopId', 'StopName', 'LineAbbr', 'transit_type', 'stop_category', 'geometry']
final_stops = pd.concat([
    major_transit[cols_common],
    unique_bus_stops[cols_common]
], ignore_index=True)

# Rename and save
final_stops = final_stops.rename(columns={
    'StopName': 'name',
    'transit_type': 'type',
    'stop_category': 'category'
})

final_stops.to_file('data/processed/septa_stops.geojson', driver='GeoJSON')
septa_gdf = final_stops.copy()

print(f"\n✓ Final dataset: {len(final_stops)} stops")
print(f"  • Major transit: {len(final_stops[final_stops['category'] == 'Major Transit'])}")
print(f"  • Connecting bus: {len(final_stops[final_stops['category'] == 'Feeder Bus'])}")
print("✓ Saved to: data/processed/septa_stops.geojson")

# Visualize
print("\nCreating visualization...")

fig, ax = plt.subplots(figsize=(16, 14))

# Study area and network
study_area.boundary.plot(ax=ax, color='blue', linewidth=3, linestyle='--', 
                         label='Study Area', zorder=1)
edges_gdf.plot(ax=ax, color='lightgray', linewidth=0.3, alpha=0.2, zorder=2)

# Major transit stations
major_plot = final_stops[final_stops['category'] == 'Major Transit']
major_plot.plot(ax=ax, color='darkred', markersize=500, marker='*',
               edgecolor='black', linewidth=2, 
               label=f'Major Transit ({len(major_plot)})', zorder=10)

# Bus stops
bus_plot = final_stops[final_stops['category'] == 'Feeder Bus']
bus_plot.plot(ax=ax, color='green', markersize=80, marker='o',
             edgecolor='darkgreen', linewidth=1,
             label=f'Connecting Bus ({len(bus_plot)})', alpha=0.7, zorder=9)

# Labels for major stations
for idx, row in major_plot.iterrows():
    ax.annotate(row['name'], xy=(row.geometry.x, row.geometry.y),
               xytext=(15, 15), textcoords='offset points',
               fontsize=10, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.6', facecolor='white',
                        edgecolor='darkred', linewidth=2, alpha=0.95),
               zorder=11)

ax.set_title(f'SEPTA Transit Stops - University City\n' + 
            f'Major Stations + Connecting Bus Stops ({len(final_stops)} total)',
            fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.legend(fontsize=12, loc='upper right', framealpha=0.95, edgecolor='black')

plt.tight_layout()
plt.savefig('outputs/maps/03_septa_stops.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Map saved to: outputs/maps/03_septa_stops.png")

print("\n" + "="*70)
print("Transit connectivity dataset ready for shade routing analysis!")
print("="*70)


LOADING OFFICIAL SEPTA STOPS DATA
Loading from: data/raw/Transit_Stops_(Spring_2025).geojson
✓ Loaded 22,478 SEPTA stops system-wide


NameError: name 'study_area' is not defined